# Handling Unwanted Topics with Semantic Router

In [1]:
import os
import json

from semantic_router import RouteLayer
from extended_route import ExtendedRoute
from semantic_router_chain import SemanticRouter
from semantic_router.encoders import HuggingFaceEncoder

In [ ]:
# installing the required packages
!pip install -r requirements.txt

## Loading the routes

In [2]:
routes = {}
path = "./routes"
for file in os.listdir(path):
    if file.endswith(".json"):
        with open(os.path.join(path, file), "r") as f:
            route = json.load(f)
            routes[route["name"]] = ExtendedRoute(route)

Using Huggingface Encoder - we will use bge-large-en-v1.5 open source model

In [3]:
model_name = "BAAI/bge-large-en-v1.5"
encoder = HuggingFaceEncoder(name=model_name)

Creating Aurelio.AI route layer instance - this is the actual router that will be used to route the requests

In [4]:
router = RouteLayer(encoder=encoder, routes=list(routes.values()))

2024-03-28 23:22:57 INFO semantic_router.utils.logger local


In [5]:
router("What do you think about buying TSLA stock?")

RouteChoice(name='investments', function_call=None, similarity_score=None)

In [6]:
router("I need to write a Python script for web scraping. Can you assist?")

RouteChoice(name='coding', function_call=None, similarity_score=None)

## Creating a Semantic Router Chain using Langchain

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from dummy_llm import DummyLLM

# creates a dummy chain for demonstration purposes
dummy_rag_chain = LLMChain(llm=DummyLLM(responses=["Hi, I'm a Dummy RAG Chain :)"]), prompt=PromptTemplate.from_template(""))

In [16]:
# creating the semantic router chain
chain = SemanticRouter.from_llm(rag_chain=dummy_rag_chain, router_layer=router)

*Note*: `rag_chain` should be replaced with a real RAG model

Let's run a three-step test, hoping the model identifies our code query and follows our scripted answers.

In [17]:
for i in range(3):
    result = chain.invoke("I need to write a Python script for web scraping. Can you assist?")['result']
    print(result, end="\n\n")

I'm afraid I can't assist with that. My expertise is in vacation recommendations, not coding inquiries. Feel free to ask anything about vacations though!

Oops! Looks like you're asking about coding. I'm the Vacation Recommendations ChatBot and my speciality lies in the domain of vacations. Try me with any query related to that and I'll do my best to assist!

My apologies, but your query seems to be about coding. As a Vacation Recommendations ChatBot, my skills are geared towards providing assistance on finding the best tips to ypur vacation. Please do ask me any question on this topic and I'll be glad to help!



Voila! We have successfully handled the unwanted topic using Semantic Router.

we can do the same thing with investing related queries

In [18]:
for i in range(3):
    result = chain.invoke("What's your take on investing in the healthcare sector?")['result']
    print(result, end="\n\n")

I'm afraid I can't provide help with investment matters. My primary role is to offer insight on vacation recommendations. Please ask me anything about those topics, and I'd be more than happy to assist.

Sorry for any misunderstanding, but I am not equipped to answer investment-related questions. My expertise lies in recommendations for the best vacation. Feel free to ask me anything in that regard!

Unfortunately, I can't assist with investment queries. As a Vacation Recommendations ChatBot assistant, I'm trained to help with questions regarding our vacations only. Please ask me any queries related to these topics!



Great! Now, let's ask for something related to the RAG app, specifically a recommendation for the best bakery in Paris.

In [19]:
result = chain.invoke("what is the best bakery in paris?")['result']
print(result)

Hi, I'm a Dummy RAG Chain :)
